In [6]:
#Analisi ESG su aziende Energy (USA/EU)

#Layer A: test di predittività -> Random Forest sui rendimenti relativi a 12 mesi (target in avanti).
#Layer B: outcome -> Sharpe ratio relativo per gruppi (High vs Low ESG), per periodo e area.

#NOTE:
# Usiamo benchmark settoriali da file (USA≈XLE; EU≈EXH1.DE) già in termini di rendimenti mensili.
#-Rimuoviamo i duplicati mensili sui prezzi (se ci sono più righe nello stesso mese, teniamo l’ultima).
#Periodi ordinati: pre_trump (2013–2016), trump (2017–2020), biden (2021–2024).


import os
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error


# PERCORSI FILE 

PRICES_CSV     = r"C:\Users\totim\OneDrive\Desktop\prezzi_mensili_aggiornato.csv"     # prezzi mensili (date,ticker,price)
BENCH_CSV      = r"C:\Users\totim\OneDrive\Desktop\benchmarks_monthly_returns.csv"    # rendimenti benchmark (date,bm_USA,bm_EU)
ESG_PANEL_XLSX = r"C:\Users\totim\OneDrive\Desktop\SALVATORE MEGNA DATI PANEL.xlsx"   # dataset ESG (mensile)


# CONFIGURAZIONE ANALISI

# Gruppi titolo 
USA_HIGH = ["NEE", "XEL", "DUK"]                # Utility/transizione (High ESG)
USA_LOW  = ["XOM", "CVX", "OXY"]                # Oil & Gas (Low ESG)
EU_HIGH  = ["ENEL.MI", "IBE.MC", "EOAN.DE"]     # Utility/transizione (High ESG)
EU_LOW   = ["TTE.PA", "BP.L", "ENI.MI"]         # Oil & Gas (Low ESG)
TICKERS_TUTTI = USA_HIGH + USA_LOW + EU_HIGH + EU_LOW

# Ordine periodi coerente in tutte le tabelle
PERIODI_ORDINATI = ["pre_trump", "trump", "biden"]

# Mappatura ticker ESG -> ticker Yahoo Finance
MAPPA_ESG2YF = {
    "NEE-US":"NEE","XEL-US":"XEL","DUK-US":"DUK",
    "XOM-US":"XOM","CVX-US":"CVX","OXY-US":"OXY",
    "ENEL-IT":"ENEL.MI","IBE-ES":"IBE.MC","EOAN-DE":"EOAN.DE",
    "TTE-FR":"TTE.PA","BP-GB":"BP.L","ENI-IT":"ENI.MI"
}

# Funzioni utilità per area e classe ESG
def _area_di(ticker: str) -> str:
    return "USA" if ticker in (USA_HIGH + USA_LOW) else "EU"

def _classe_esg_di(ticker: str) -> str:
    return "High" if ticker in (USA_HIGH + EU_HIGH) else "Low"


# CARICAMENTO E PREPARAZIONE DATI

def carica_prezzi_e_rendimenti(percorso_csv: str) -> pd.DataFrame:
    """
    Legge prezzi mensili (date, ticker, price) e calcola i rendimenti mensili per ticker (wide).
    FIX duplicati: se nello stesso mese esistono più righe per un ticker, tiene SOLO l’ultima del mese.
    Ritorna un DataFrame wide di rendimenti (index=date month-end, colonne=tickers).
    """
    if not os.path.exists(percorso_csv):
        raise FileNotFoundError(f"File prezzi non trovato: {percorso_csv}")

    df = pd.read_csv(percorso_csv)
    richieste = {"date", "ticker", "price"}
    if not richieste.issubset(df.columns):
        raise ValueError(f"Colonne richieste per prezzi: {richieste}. Trovate: {df.columns.tolist()}")

    # filtra i ticker del progetto
    df = df[df["ticker"].isin(TICKERS_TUTTI)].copy()
    if df.empty:
        raise ValueError("Il file prezzi non contiene i ticker richiesti dal progetto.")

    # forza date a month-end e prendi l’ultima osservazione del mese per (ticker, mese)
    df["date"] = pd.to_datetime(df["date"]).dt.to_period("M").dt.to_timestamp("M")
    df = (df.sort_values(["ticker", "date"])
            .groupby(["ticker", "date"], as_index=False)["price"]
            .last())

    # pivot e rendimenti mensili
    prezzi_wide = df.pivot(index="date", columns="ticker", values="price").sort_index()
    r_titoli = prezzi_wide.pct_change().dropna(how="all")

    print(">>> Controllo duplicati su date di r_titoli:", int(r_titoli.index.duplicated().sum()))
    return r_titoli


def carica_benchmark(percorso_csv: str) -> tuple[pd.Series, pd.Series]:
    """
    Carica i rendimenti mensili dei benchmark di settore da file (già in % mensile).
    Restituisce due Serie indicizzate a month-end: (bm_USA, bm_EU).
    """
    if not os.path.exists(percorso_csv):
        raise FileNotFoundError(f"File benchmark non trovato: {percorso_csv}")

    bm = pd.read_csv(percorso_csv)
    richieste = {"date", "bm_USA", "bm_EU"}
    if not richieste.issubset(bm.columns):
        raise ValueError(f"Colonne richieste per benchmark: {richieste}. Trovate: {bm.columns.tolist()}")

    bm["date"] = pd.to_datetime(bm["date"]).dt.to_period("M").dt.to_timestamp("M")
    bm = bm.sort_values("date").set_index("date")

    return bm["bm_USA"].astype(float), bm["bm_EU"].astype(float)


def carica_esg_lag1(percorso_xlsx: str) -> pd.DataFrame:
    """
    Carica il dataset ESG, mappa i ticker a Yahoo, ordina per (ticker,data),
    e costruisce i punteggi a lag 1 (mese precedente) per E, S, G e totale ESG.
    Ritorna un DataFrame con colonne: ticker, date, E_lag1, S_lag1, G_lag1, ESG_lag1.
    """
    if not os.path.exists(percorso_xlsx):
        raise FileNotFoundError(f"File ESG non trovato: {percorso_xlsx}")

    esg = pd.read_excel(percorso_xlsx, sheet_name=0)

    # parsing data (mm/yy) -> month-end
    esg["date"] = pd.to_datetime(esg["Date"], format="%m/%y").dt.to_period("M").dt.to_timestamp("M")
    esg["ticker"] = esg["Ticker"].map(MAPPA_ESG2YF)

    esg = esg.dropna(subset=["ticker"]).sort_values(["ticker", "date"])

    # lag mensile
    esg["E_lag1"]   = esg.groupby("ticker")["ESG-E SCORE"].shift(1)
    esg["S_lag1"]   = esg.groupby("ticker")["ESG-S SCORE"].shift(1)
    esg["G_lag1"]   = esg.groupby("ticker")["ESG-G SCORE"].shift(1)
    esg["ESG_lag1"] = esg.groupby("ticker")["ESG TOT SCORE"].shift(1)

    esg = esg[["ticker", "date", "E_lag1", "S_lag1", "G_lag1", "ESG_lag1"]].dropna()
    return esg[esg["ticker"].isin(TICKERS_TUTTI)]


def costruisci_rendimenti_relativi(r_titoli: pd.DataFrame,
                                   bm_usa: pd.Series,
                                   bm_eu: pd.Series) -> pd.DataFrame:
    """
    Costruisce i rendimenti relativi: r_rel = r_titolo - r_benchmark_area
    dove l’area è 'USA' o 'EU' a seconda del ticker.
    Restituisce un DataFrame long con: date, ticker, area, r_stock, r_sector, r_rel.
    """
    # unisco i benchmark in una tabella (index = date)
    bm = pd.DataFrame({"USA": bm_usa, "EU": bm_eu}).dropna(how="any")

    # allineo le date dei rendimenti dei titoli a quelle dei benchmark
    r_titoli = r_titoli.loc[bm.index]

    # formato “long”
    lungo = r_titoli.stack().reset_index()
    lungo.columns = ["date", "ticker", "r_stock"]
    lungo["area"] = lungo["ticker"].map(_area_di)
    lungo["r_sector"] = lungo.apply(lambda r: bm.loc[r["date"], r["area"]], axis=1)
    lungo["r_rel"] = lungo["r_stock"] - lungo["r_sector"]

    # sicurezza: date a month-end
    lungo["date"] = lungo["date"].dt.to_period("M").dt.to_timestamp("M")
    return lungo.dropna(subset=["r_rel"])


def aggiungi_colonne_periodo(df: pd.DataFrame) -> pd.DataFrame:
    """
    Aggiunge colonne:
    - periodo (pre_trump, trump, biden) con ordine imposto
    - area (USA/EU)
    - esg_class (High/Low)
    Rimuove eventuali righe fuori intervallo analizzato.
    """
    df = df.copy()
    df["periodo"] = pd.Categorical(
        np.select(
            [
                (df["date"] >= pd.Timestamp("2013-01-01")) & (df["date"] <= pd.Timestamp("2016-12-31")),
                (df["date"] >= pd.Timestamp("2017-01-01")) & (df["date"] <= pd.Timestamp("2020-12-31")),
                (df["date"] >= pd.Timestamp("2021-01-01")) & (df["date"] <= pd.Timestamp("2024-12-31")),
            ],
            PERIODI_ORDINATI,
            default="out"
        ),
        categories=PERIODI_ORDINATI,
        ordered=True
    )
    df = df[df["periodo"] != "out"]
    df["area"] = df["ticker"].map(_area_di)
    df["esg_class"] = df["ticker"].map(_classe_esg_di)
    return df


# LAYER A — TARGET A 12 MESI (IN AVANTI) E RANDOM FOREST

def costruisci_target_12m_avanti(panel: pd.DataFrame) -> pd.DataFrame:
    """
    Costruisce il target a 12 mesi in avanti: somma dei 12 r_rel successivi.
    Restituisce il panel con colonna 'r_rel_12m_fwd' e senza NaN su questa colonna.
    """
    df = panel.sort_values(["ticker", "date"]).copy()
    s = 0
    for k in range(1, 13):
        s += df.groupby("ticker")["r_rel"].shift(-k)
    df["r_rel_12m_fwd"] = s
    return df.dropna(subset=["r_rel_12m_fwd"])


def modello_layer_a_target_12m(panel_12m: pd.DataFrame,
                               features: list[str] | None = None) -> pd.DataFrame:
    """
    Allena e valuta un Random Forest per periodo→area, con split temporale 80/20.
    Restituisce tabella con R2_test, MAE_test e dimensioni campione per blocco.
    """
    if features is None:
        features = ["E_lag1", "S_lag1", "G_lag1", "ESG_lag1"]

    righe = []
    for periodo in PERIODI_ORDINATI:      # coerenza: prima periodo
        for area in ["USA", "EU"]:        # poi area
            d = panel_12m[(panel_12m["periodo"] == periodo) & (panel_12m["area"] == area)].copy()
            if len(d) < 40:
                continue

            d = d.sort_values("date")
            split = int(len(d) * 0.8)     # train/test temporale

            Xtr, Xte = d[features].iloc[:split], d[features].iloc[split:]
            ytr, yte = d["r_rel_12m_fwd"].iloc[:split], d["r_rel_12m_fwd"].iloc[split:]

            rf = RandomForestRegressor(n_estimators=400, random_state=42, n_jobs=-1)
            rf.fit(Xtr, ytr)
            y_pred = rf.predict(Xte)

            righe.append({
                "periodo": periodo,
                "area": area,
                "R2_test": r2_score(yte, y_pred),
                "MAE_test": mean_absolute_error(yte, y_pred),
                "n_total": len(d),
                "n_train": len(Xtr),
                "n_test": len(Xte)
            })

    out = pd.DataFrame(righe).round(3)
    if not out.empty:
        out["periodo"] = pd.Categorical(out["periodo"], PERIODI_ORDINATI, ordered=True)
        out = out.sort_values(["periodo", "area"])
    return out


# LAYER B — SHARPE AGGREGATO E BOOTSTRAP DELTA (HIGH−LOW)

def sharpe_aggregato_per_gruppo(panel: pd.DataFrame) -> pd.DataFrame:
    """
    Per ogni (periodo × area × classe ESG) calcola:
    - serie equal-weight dei r_rel dei titoli del gruppo (media trasversale)
    - Sharpe annualizzato = (media/std) * sqrt(12)
    Riporta anche media, deviazione standard e numero di mesi.
    """
    righe = []
    for (per, area, cls), g in panel.groupby(["periodo", "area", "esg_class"]):
        serie = (g.pivot_table(index="date", columns="ticker", values="r_rel")
                 .mean(axis=1).dropna())
        if serie.shape[0] < 12:
            continue
        mu, sd = serie.mean(), serie.std(ddof=1)
        righe.append({
            "periodo": per,
            "area": area,
            "esg_class": cls,
            "sharpe_agg": (mu / sd) * np.sqrt(12) if sd > 0 else np.nan,
            "mean": mu,
            "std": sd,
            "n": serie.shape[0]
        })

    out = pd.DataFrame(righe).round(3)
    if not out.empty:
        out["periodo"] = pd.Categorical(out["periodo"], PERIODI_ORDINATI, ordered=True)
        out = out.sort_values(["periodo", "area", "esg_class"])
    return out


def bootstrap_delta_sharpe(panel: pd.DataFrame,
                           n_boot: int = 5000,
                           seed: int = 42) -> pd.DataFrame:
    """
    Confronto High vs Low ESG per (periodo × area) sullo Sharpe aggregato:
    - delta = Sharpe(High) − Sharpe(Low)
    - intervallo di confidenza 95% e p-value bootstrap (resampling temporale)
    """
    rng = np.random.default_rng(seed)
    righe = []

    for (per, area), g in panel.groupby(["periodo", "area"]):
        high = (g[g["esg_class"] == "High"]
                .pivot_table(index="date", columns="ticker", values="r_rel")
                .mean(axis=1).dropna())
        low  = (g[g["esg_class"] == "Low"]
                .pivot_table(index="date", columns="ticker", values="r_rel")
                .mean(axis=1).dropna())

        idx_comuni = high.index.intersection(low.index)
        if len(idx_comuni) < 12:
            continue

        h = high.loc[idx_comuni].values
        l = low.loc[idx_comuni].values

        def sharpe(x: np.ndarray) -> float:
            sd = x.std(ddof=1)
            return (x.mean() / sd) * np.sqrt(12) if sd > 0 else np.nan

        delta = sharpe(h) - sharpe(l)

        # bootstrap temporale (resampling con rimpiazzo sugli indici 0..T-1)
        T = len(idx_comuni)
        boots = []
        for _ in range(n_boot):
            b = rng.integers(0, T, T)
            boots.append(sharpe(h[b]) - sharpe(l[b]))
        boots = np.array(boots)

        ci_lo, ci_hi = np.quantile(boots, [0.025, 0.975])
        pval = (np.mean(boots >= abs(delta)) + np.mean(boots <= -abs(delta)))
        pval = min(1.0, pval)

        righe.append({
            "periodo": per,
            "area": area,
            "delta_sharpe_high_minus_low": delta,
            "ci95_low": ci_lo,
            "ci95_high": ci_hi,
            "p_value": pval,
            "n_months": T
        })

    out = pd.DataFrame(righe).round(3)
    if not out.empty:
        out["periodo"] = pd.Categorical(out["periodo"], PERIODI_ORDINATI, ordered=True)
        out = out.sort_values(["periodo", "area"])
    return out


# PROGRAMMA PRINCIPALE

def main() -> None:
    print("=" * 78)
    print("ANALISI ESG ENERGY — avvio (Layer A 12M → Layer B)")
    print("=" * 78)

    # 1) Rendimenti mensili dei titoli, con fix duplicati
    r_titoli = carica_prezzi_e_rendimenti(PRICES_CSV)

    # 2) Rendimenti benchmark (USA/EU) da file
    bm_usa, bm_eu = carica_benchmark(BENCH_CSV)

    # 3) dataset base con rendimenti relativi
    base_rel = costruisci_rendimenti_relativi(r_titoli, bm_usa, bm_eu)

    # 4) Punteggi ESG (lag1) e arricchimento panel con periodo/area/classe
    esg_lag = carica_esg_lag1(ESG_PANEL_XLSX)
    panel = aggiungi_colonne_periodo(base_rel.merge(esg_lag, on=["ticker", "date"], how="inner"))

    #  Anteprima pulita (senza r_rel) 
    colonne_preview = ["date","ticker","area","periodo","esg_class","E_lag1","S_lag1","G_lag1","ESG_lag1"]
    print("\n>>> Anteprima PANEL (senza r_rel):")
    print(panel[colonne_preview].head().to_string(index=False))

    # Info dataset
    print("\nTickers nel panel:", sorted(panel["ticker"].unique()))
    print("Date:", panel["date"].min().date(), "→", panel["date"].max().date())
    conteggi = (panel.groupby(["periodo","area","esg_class"])["ticker"]
                .count().rename("n_osservazioni").reset_index())
    conteggi["periodo"] = pd.Categorical(conteggi["periodo"], PERIODI_ORDINATI, ordered=True)
    conteggi = conteggi.sort_values(["periodo","area","esg_class"])
    print("\nDistribuzione osservazioni (periodo → area → classe):")
    print(conteggi.to_string(index=False))

    #  LAYER A 
    panel_12m = costruisci_target_12m_avanti(panel)
    print("\n=== TARGET 12M — Check (senza r_rel) ===")
    cols_chk = ["ticker","date","r_rel_12m_fwd","E_lag1","S_lag1","G_lag1","ESG_lag1","periodo","area","esg_class"]
    print(panel_12m[cols_chk].head().to_string(index=False))

    metriche12m = modello_layer_a_target_12m(panel_12m)
    print("\n=== LAYER A (12M avanti) — METRICHE TEST (periodo → area) ===")
    cols_out = ["periodo","area","R2_test","MAE_test","n_total","n_train","n_test"]
    print(metriche12m[cols_out].to_string(index=False) if not metriche12m.empty else "Dati insufficienti.")

    # LAYER B 
    sharpe_gruppi = sharpe_aggregato_per_gruppo(panel)
    print("\n=== SHARPE RELATIVO — AGGREGATO per GRUPPO (periodo → area × classe) ===")
    print(sharpe_gruppi.to_string(index=False) if not sharpe_gruppi.empty else "Dati insufficienti.")

    boot = bootstrap_delta_sharpe(panel)
    print("\n=== TEST BOOTSTRAP: Delta Sharpe (High - Low) (periodo → area) ===")
    print(boot.to_string(index=False) if not boot.empty else "Dati insufficienti.")

    print("\nFINE.")


if __name__ == "__main__":
    main()

ANALISI ESG ENERGY — avvio (Layer A 12M → Layer B)
>>> Controllo duplicati su date di r_titoli: 0

>>> Anteprima PANEL (senza r_rel):
      date  ticker area   periodo esg_class    E_lag1    S_lag1    G_lag1  ESG_lag1
2013-02-28    BP.L   EU pre_trump       Low 35.858662 33.591288 32.297575 34.039327
2013-02-28     CVX  USA pre_trump       Low 48.812464 42.392669 34.117823 41.860691
2013-02-28     DUK  USA pre_trump      High 67.009163 44.420507 44.554772 64.186761
2013-02-28 ENEL.MI   EU pre_trump      High 77.691854 45.629006 36.938903 72.563546
2013-02-28  ENI.MI   EU pre_trump       Low 76.225094 29.708024 29.087768 35.850078

Tickers nel panel: ['BP.L', 'CVX', 'DUK', 'ENEL.MI', 'ENI.MI', 'EOAN.DE', 'IBE.MC', 'NEE', 'OXY', 'TTE.PA', 'XEL', 'XOM']
Date: 2013-02-28 → 2024-12-31

Distribuzione osservazioni (periodo → area → classe):
  periodo area esg_class  n_osservazioni
pre_trump   EU      High             141
pre_trump   EU       Low             141
pre_trump  USA      High       